In [ ]:
kwargs = {
    "seed": 42,
    "data_dir": "data/",
    "train_dir": "outputs/multi_task_model",
    "model_file": "outputs/pytorch_model.bin",
    "model_id": "1kZ8RmDj8K3HihmUiW2gJu8iyZ82cpbex",
    "num_past_utterances": 10,
    "num_future_utterances": 0,
    "speaker_in_context": False,
    "epoch": 6,
    "learning_rate": 1e-5,
    "batch_size": 8,
    "do_train": False,
    "checkpoint": "markussagen/xlm-roberta-longformer-base-4096",
    "train_dataset": ["MPDD", "MELD", "EmoryNLP"],
    "train_task": ["Emotion"],
    "eval_dataset": "MPDD",
    "eval_task": "Emotion",
    'output_file': 'outputs/predictions.out',
    "result_file": "results/scores.out",
}

###Initialize

In [ ]:
from google.colab import drive
if kwargs["do_train"]:
  drive.mount('/content/gdrive')

In [ ]:
!mkdir -p data/MELD
!cd data/MELD && wget https://raw.githubusercontent.com/UW-ling573-2022/data/main/MELD/dev_sent_emo.csv
!cd data/MELD && wget https://raw.githubusercontent.com/UW-ling573-2022/data/main/MELD/test_sent_emo.csv
!cd data/MELD && wget https://raw.githubusercontent.com/UW-ling573-2022/data/main/MELD/train_sent_emo.csv
!mkdir -p data/EMORYNLP
!cd data/EMORYNLP && wget https://raw.githubusercontent.com/UW-ling573-2022/data/main/EMORYNLP/dev.csv
!cd data/EMORYNLP && wget https://raw.githubusercontent.com/UW-ling573-2022/data/main/EMORYNLP/train.csv
!cd data/EMORYNLP && wget https://raw.githubusercontent.com/UW-ling573-2022/data/main/EMORYNLP/test.csv
!mkdir -p data/MPDD
!cd data/MPDD && wget https://raw.githubusercontent.com/UW-ling573-2022/data/main/MPDD/dev_mpdd.csv
!cd data/MPDD && wget https://raw.githubusercontent.com/UW-ling573-2022/data/main/MPDD/train_mpdd.csv
!cd data/MPDD && wget https://raw.githubusercontent.com/UW-ling573-2022/data/main/MPDD/test_mpdd.csv

In [ ]:
!mkdir -p outputs
import gdown
if kwargs["model_id"] and not kwargs["do_train"]:
  gdown.download(id=kwargs["model_id"], output=kwargs["model_file"], quiet=False)

In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece

In [ ]:
import datasets
import numpy as np
import torch
import torch.nn as nn
from datasets import ClassLabel, load_metric, Dataset
from datasets import load_dataset, concatenate_datasets
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments
from transformers import PreTrainedModel, PretrainedConfig
from transformers import Trainer
from transformers import is_datasets_available
from transformers.trainer_pt_utils import IterableDatasetShard

###MTL/data.py

In [ ]:
class SingleTaskDataLoader:
    def __init__(self, task, **kwargs):
        self.task = task
        self.data_loader = DataLoader(**kwargs)
        self.batch_size = self.data_loader.batch_size
        self.dataset = self.data_loader.dataset
        
    def __len__(self) -> int:
        return len(self.data_loader)
    
    def __iter__(self):
        for batch in self.data_loader:
            batch["task"] = self.task
            yield batch
    
class MultiTaskDataLoader:
    def __init__(self, task_data_loaders):
        self.task_data_loaders = task_data_loaders
        self.dataset = [None] * sum([len(dl.dataset) for dl in task_data_loaders.values()])
        
    def __len__(self) -> int:
        return sum([len(dl) for dl in self.task_data_loaders.values()])
    
    def __iter__(self):
        task_choices = []
        for task, dl in self.task_data_loaders.items():
            task_choices.extend([task] * len(dl))
        task_choices = np.array(task_choices)
        np.random.shuffle(task_choices)
        for task in task_choices:
            yield next(iter(self.task_data_loaders[task]))

###MTL/model.py

In [ ]:
class MultiTaskModel(PreTrainedModel):
    def __init__(self, encoder, task_models):
        super(MultiTaskModel, self).__init__(PretrainedConfig())
        self.encoder = encoder
        self.task_models = nn.ModuleDict(task_models)
        
    @classmethod
    def from_task_models(cls, task_models):
        shared_encoder = None
        for model in task_models.values():
            if shared_encoder is None:
                shared_encoder = getattr(model, cls.get_encoder_attr_name(model))
            else:
                setattr(model, cls.get_encoder_attr_name(model), shared_encoder)
        return cls(shared_encoder, task_models)
                  
    @staticmethod
    def get_encoder_attr_name(model):
        model_name = model.__class__.__name__
        if model_name.startswith('Bert'):
            return 'bert'
        elif model_name.startswith('Roberta') or model_name.startswith("XLMRoberta"):
            return 'roberta'
        elif model_name.startswith('Albert'):
            return 'albert'
        else:
            raise ValueError('Unsupported model: {}'.format(model_name))
        
    def forward(self, task, input_ids, attention_mask, **kwargs):
        model = self.task_models[task]
        return model(input_ids, attention_mask, **kwargs)

###MLT/train.py

In [ ]:
class MultiTaskTrainer(Trainer):

    def get_single_task_dataloader(self, task, dataset, description):
        if description == "training" and self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        elif description == "evaluation" and dataset is None and self.eval_dataset is None:
            raise ValueError("Trainer: evaluation requires an eval_dataset.")

        if is_datasets_available() and isinstance(dataset, Dataset):
            dataset = self._remove_unused_columns(dataset, description=description)

        if isinstance(dataset, torch.utils.data.IterableDataset):
            if self.args.world_size > 1:
                dataset = IterableDatasetShard(
                    dataset,
                    batch_size=self.args.train_batch_size,
                    drop_last=self.args.dataloader_drop_last,
                    num_processes=self.args.world_size,
                    process_index=self.args.process_index,
                )

            return SingleTaskDataLoader(
                task,
                dataset=dataset,
                batch_size=self.args.per_device_train_batch_size,
                collate_fn=self.data_collator,
                num_workers=self.args.dataloader_num_workers,
                pin_memory=self.args.dataloader_pin_memory,
            )

        if description == "training":
            self.train_dataset, dataset = dataset, self.train_dataset
            sampler = self._get_train_sampler()
            self.train_dataset, dataset = dataset, self.train_dataset
            batch_size = self.args.train_batch_size
        else:
            sampler = self._get_eval_sampler(dataset)
            batch_size = self.args.eval_batch_size

        return SingleTaskDataLoader(
            task,
            dataset=dataset,
            batch_size=batch_size,
            sampler=sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )

    def get_train_dataloader(self):
        return MultiTaskDataLoader({
            task: self.get_single_task_dataloader(task, dataset, description="training")
            for task, dataset in self.train_dataset.items()
        })

    def get_eval_dataloader(self, eval_dataset=None):
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        task_to_eval = eval_dataset["task"]
        return self.get_single_task_dataloader(task_to_eval, eval_dataset[task_to_eval], description="evaluation")

    def get_test_dataloader(self, test_dataset):
        task_to_test = test_dataset["task"]
        return self.get_single_task_dataloader(task_to_test, test_dataset[task_to_test], description="test")


###preprocess.py

In [ ]:
def preprocess(tokenizer, dataset_labels, **kwargs):

    meld_files = {
        "train": kwargs["data_dir"] + "MELD/train_sent_emo.csv", 
        "validation": kwargs["data_dir"] + "MELD/dev_sent_emo.csv",
        "test": kwargs["data_dir"] + "MELD/test_sent_emo.csv"
    }
    
    emorynlp_files = {
        "train": kwargs["data_dir"] + "EMORYNLP/train.csv",
        "validation": kwargs["data_dir"] + "EMORYNLP/dev.csv",
        "test": kwargs["data_dir"] + "EMORYNLP/test.csv"
    }
    
    mpdd_files = {
        "train": kwargs["data_dir"] + "MPDD/train_mpdd.csv", 
        "validation": kwargs["data_dir"] + "MPDD/dev_mpdd.csv",
        "test": kwargs["data_dir"] + "MPDD/test_mpdd.csv"
    }
    
    datasets = {"MELD": load_dataset("csv", data_files=meld_files),
                "EmoryNLP": load_dataset("csv", data_files=emorynlp_files),
                "MPDD": load_dataset("csv", data_files=mpdd_files)}
    
    def encode_label(example, labels):
        for task, label in labels.items():
            if task == "Speaker":
                example[task] = label.str2int(example[task]) \
                    if example[task] in label.names else label.str2int("Others")
            else:
                example[task] = label.str2int(example[task])
        return example

    for name, dataset in datasets.items():
        datasets[name] = dataset.map(lambda e: encode_label(e, dataset_labels[name]))

    def add_context(example, idx, dataset, labels):
        example["Past"] = ""
        example["Future"] = ""

        if example["Utterance_ID"] != 0:
            i = 1
            while idx - i >= 0:
                past = dataset[idx - i]
                past_utterance = past["Utterance"]
                if "Speaker" in labels and kwargs["speaker_in_context"]:
                    past_speaker = labels["Speaker"].int2str(past["Speaker"])
                    example["Past"] = past_speaker + ":" + past_utterance + " " + example["Past"]
                else:
                    example["Past"] = past_utterance + " " + example["Past"]
                if past["Utterance_ID"] == 0 or i >= kwargs["num_past_utterances"]:
                    break
                i += 1

        if idx + 1 < len(dataset) and dataset[idx + 1]["Utterance_ID"] != 0:
            i = 1
            while idx + i < len(dataset):
                future = dataset[idx + i]
                future_utterance = future["Utterance"]
                if "Speaker" in labels and kwargs["speaker_in_context"]:
                    future_speaker = labels["Speaker"].int2str(future["Speaker"])
                    example["Future"] += " " + future_speaker + ":" + future_utterance
                else:
                    example["Future"] += " " + future_utterance
                i += 1
                if idx + i < len(dataset) and dataset[idx + i]["Utterance_ID"] == 0 \
                    or i >= kwargs["num_future_utterances"]:
                    break

        return example

    for name, dataset in datasets.items():
        for split, ds in dataset.items():
            dataset[split] = ds.map(lambda e, i: add_context(e, i, ds, dataset_labels[name]), with_indices=True)

    def tokenize(example, add_past, add_future):
        if add_past:
            return tokenizer(example["Past"], example["Utterance"])
        elif add_future:
            return tokenizer(example["Utterance"], example["Future"])
        else:
            return tokenizer(example["Utterance"])
        
    for name, dataset in datasets.items():
        cx_datasets = {}
        cx_datasets["with_past"] = dataset.map(
            lambda e: tokenize(e, add_past=True, add_future=False), batched=True)
        cx_datasets["with_future"] = dataset.map(
            lambda e: tokenize(e, add_past=False, add_future=True), batched=True)
        cx_datasets["no_context"] = dataset.map(
            lambda e: tokenize(e, add_past=False, add_future=False), batched=True)

        tasks = list(dataset_labels[name].keys())
        for cx in cx_datasets:
            cols_to_keep = ["input_ids", "attention_mask"] + tasks
            cols_to_remove = [c for c in cx_datasets[cx]["train"].column_names if c not in cols_to_keep]
            cx_datasets[cx] = cx_datasets[cx].remove_columns(cols_to_remove)
            task_datasets = {}
            for task in tasks:
                label = dataset_labels[name][task]
                ds = cx_datasets[cx]
                ds = ds.cast_column(task, label)
                ds = ds.remove_columns([t for t in tasks if t != task])
                ds = ds.rename_column(task, "labels")
                ds.set_format()
                task_datasets[task] = (ds, label)
            cx_datasets[cx] = task_datasets
        datasets[name] = cx_datasets

    return datasets


###pipeline.py

In [ ]:
def prepare_datasets(datasets, **kwargs):
    for dataset_name, cx_datasets in datasets.items():
        task_dataset = {}
        for split in ["train", "validation", "test"]:
            task_dataset[split] = {}
            for cx in cx_datasets:
                if cx == "with_past" and kwargs["num_past_utterances"] == 0:
                    continue
                elif cx == "with_future" and kwargs["num_future_utterances"] == 0:
                    continue
                elif cx == "no_context" and kwargs["num_past_utterances"] + kwargs["num_future_utterances"] > 0:
                    continue
                else:
                    for task, (ds, _) in cx_datasets[cx].items():
                        if split == "train" and task not in kwargs["train_task"]:
                            continue
                        if task not in task_dataset[split]:
                            task_dataset[split][task] = ds[split]
                        else:
                            ds_to_concat = [task_dataset[split][task], ds[split]]
                            task_dataset[split][task] = concatenate_datasets(ds_to_concat)

        train_dataset = task_dataset["train"]
        eval_dataset = task_dataset["validation"]
        test_dataset = task_dataset["test"]
        
        datasets[dataset_name] = {"train": train_dataset, "validation": eval_dataset, "test": test_dataset}
        
    train_dataset = {dataset_name + "_" + task: datasets[dataset_name]["train"][task] 
                     for dataset_name in datasets if dataset_name in kwargs["train_dataset"]
                     for task in datasets[dataset_name]["train"] if task in kwargs["train_task"]}

    eval_dataset_task = kwargs["eval_dataset"] + "_" + kwargs["eval_task"]
    eval_dataset = {eval_dataset_task: datasets[kwargs["eval_dataset"]]["validation"][kwargs["eval_task"]]}
    eval_dataset["task"] = eval_dataset_task
    test_dataset = {eval_dataset_task: datasets[kwargs["eval_dataset"]]["test"][kwargs["eval_task"]]}
    test_dataset["task"] = eval_dataset_task

    return train_dataset, eval_dataset, test_dataset


In [ ]:
dataset_labels = {
    "MELD": 
    {
        "Speaker": ClassLabel(
            num_classes=7,
            names=["Chandler", "Joey", "Monica", "Rachel", "Ross", "Phoebe", "Others"]),
        "Emotion": ClassLabel(
            num_classes=7,
            names=["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]),
        "Sentiment": ClassLabel(
            num_classes=3,
            names=["positive", "neutral", "negative"])
    },
    "EmoryNLP": 
    {   
        "Speaker": ClassLabel(
            num_classes=7,
            names=["Chandler", "Joey", "Monica", "Rachel", "Ross", "Phoebe", "Others"]),
        "Emotion": ClassLabel(
            num_classes=7,
            names=["Sad", "Mad", "Scared", "Powerful", "Peaceful", "Joyful", "Neutral"])
    },
    "MPDD": 
    {
        "Emotion": ClassLabel(
            num_classes=7,
            names=["angry", "disgust", "fear", "joy", "neutral", "sadness", "surprise"])
    },
}
    
tokenizer = AutoTokenizer.from_pretrained(kwargs["checkpoint"])
datasets = preprocess(tokenizer, dataset_labels, **kwargs)
train_dataset, eval_dataset, test_dataset = prepare_datasets(datasets, **kwargs)

In [ ]:
tasks = {task: dataset_labels[task.split("_")[0]][task.split("_")[1]] for task in train_dataset.keys()}
task_models = {
        task: AutoModelForSequenceClassification.from_pretrained(
            kwargs["checkpoint"], 
            num_labels=label.num_classes,
            max_length=1024)
        for task, label in tasks.items()
}
multi_task_model = MultiTaskModel.from_task_models(task_models)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
if not kwargs["do_train"]:
    multi_task_model.load_state_dict(
        torch.load(kwargs["model_file"], map_location=torch.device(device)))

In [ ]:
multi_task_model.to(device)

In [ ]:
def compute_metrics(eval_preds):
    metric = load_metric("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")


training_args = TrainingArguments(
    output_dir=kwargs["train_dir"],
    seed=kwargs["seed"],
    overwrite_output_dir=True,
    label_names=["labels"],
    learning_rate=kwargs["learning_rate"],
    num_train_epochs=kwargs["epoch"],
    per_device_train_batch_size=kwargs["batch_size"],
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1"
)

trainer = MultiTaskTrainer(
    multi_task_model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
if kwargs["do_train"]:
    trainer.train()

In [ ]:
pred = trainer.predict(test_dataset)
f1 = pred.metrics['test_f1']
print("Weighted F1:", f1)

In [ ]:
pred_labels = dataset_labels[kwargs["eval_dataset"]][kwargs["eval_task"]].int2str(pred.predictions.argmax(axis=-1))
true_labels = dataset_labels[kwargs["eval_dataset"]][kwargs["eval_task"]].int2str(pred.label_ids)
inputs = tokenizer.batch_decode(test_dataset[kwargs["eval_dataset"] + "_" + kwargs["eval_task"]]["input_ids"])
f = open(kwargs["output_file"], "w")
f.write("Input\tPredicted\tTrue\n")
f.write("\n".join(["\t".join([input, pred_label, true_label]) 
                    for input, pred_label, true_label 
                    in zip(inputs, pred_labels, true_labels)]))
f.close()

In [ ]:
!mkdir -p results
import json

f = open(kwargs["result_file"], "a+")
f.write(json.dumps(kwargs))
f.write("\nWeighted F1: {}\n".format(f1))
f.close()

In [ ]:
from google.colab import files
files.download(kwargs["result_file"])
files.download(kwargs["output_file"])

In [ ]:
trainer.save_model()

In [ ]:
!cp "/content/outputs/multi_task_model/pytorch_model.bin" "/content/gdrive/MyDrive/pytorch_model_en.bin"